In [ ]:
import os
import pickle
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgba2rgb, gray2rgb
import threading

In [ ]:
def classify_image_with_unknown(image_path, model, threshold=0.7):
   
    img = imread(image_path)
    ##RGB conversion
    if len(img.shape) == 2:
        img = gray2rgb(img)
    elif img.shape[-1] == 4:
        img = rgba2rgb(img)

    #Prediction
    img = resize(img, img_size, anti_aliasing=True).flatten()
    probabilities = model.predict_proba(img.reshape(1, -1))
    max_confidence = np.max(probabilities)
    
    return model.predict(img.reshape(1, -1))[0] if max_confidence >= threshold else "unknown"

In [ ]:
def test(test_image_path, test_results, best_estimator):
    if not os.path.exists(test_image_path):
        print(f"Error: The directory '{test_image_path}' does not exist.")
        exit()

    for file in os.listdir(test_image_path):
        image_path = os.path.join(test_image_path, file)
        if os.path.isfile(image_path) and file.endswith(('.png', '.jpg', '.jpeg')):
            # Classify the image
            result = classify_image_with_unknown(image_path, best_estimator, threshold=0.7)
            test_results[file] = result
            print(f"Image: {file}, Classification Result: {result}")
        else:
            print(f"Skipping {file}: Not a valid image file.")

In [ ]:
from flask import Flask, jsonify
app = Flask(__name__)

In [ ]:
# Loading the trained model
img_size = (15, 15)

model_file_name = r"classifier"
with open(model_file_name, 'rb') as file:
    best_estimator = pickle.load(file)

test_image_path = r"../../traffic_light_data/test"
test_results = {}

test(test_image_path, test_results, best_estimator)

In [ ]:
# Loading calssification_report
with open('Scripts/Resources/classification_report', 'rb') as f:
    report = pickle.load(f)

In [ ]:
def run_flask():
    app.run(host='0.0.0.0', debug=False, port=5000)

In [ ]:
@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(report), 200

In [ ]:
threading.Thread(target=run_flask, daemon=True).start()